In [78]:
import pandas as pd
import scipy.stats as stats 
import numpy as np
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_categorical_dtype
from pandas.api.types import is_string_dtype
import math

In [127]:
agey = pd.DataFrame({'age':pd.Series([21,22,24,25,27,27,27,35,41]), 'grade': pd.Series(list('FFPFPPPPP'))})
agey

wah = pd.DataFrame({'hours_studied': pd.Series([4,5,9,12,15]), 'grade': list('NYNYY')})
wah
# real = pd.read_csv("Datasets/diamonds.csv")
# real.head()

,hours_studied,grade
0,4,N
1,5,Y
2,9,N
3,12,Y
4,15,Y


True

In [136]:
def entropy_discretize(df, discretize_column, class_col ,nbins = None, threshold = None):
    probs_df =  calculate_prob(df[discretize_column])
    entropy_df = stats.entropy(probs_df, base=2)
    
    exit_cond = True
    best_boundaries = []
    x = [df]
    resey = []
    
    while exit_cond:
        
        if len(resey) > 0 : 
            x[:] = resey[:]
            resey[:] = []
        
        for a in x:
            boundaries = calculate_boundaries(a, discretize_column)
            splits = [(a.iloc[0:x+1,:], a.iloc[x+1:,:], y) for x,y in enumerate(boundaries)]
            gains = []
            for s1,s2, boundary in splits:
                probs_s1 =  calculate_prob(s1[class_col])
                probs_s2 =  calculate_prob(s2[class_col])
                entropy_s1 = stats.entropy(probs_s1, base=2)
                entropy_s2 = stats.entropy(probs_s2, base=2)
                total_size = df.shape[0]
                gains.append((info_gain(entropy_s1,entropy_s2,s1,s2,total_size) , boundary,s1, s2))
#             print(gains)
            max_info = min(gains,  key = lambda t: t[0])
    
            best_boundaries.append(max_info)
            # took the most simplest path dunt have time for optimization
            resey.append(max_info[2])
            resey.append(max_info[3])
            # stopping condition
            if nbins:
                if len(best_boundaries) * 2 >= nbins:
                    exit_cond = False
            elif threshold:
                if entropy_df - max_info[0] > threshold :
                    exit_cond = False
            
                
    return best_boundaries
            
        

    
    
#     for best_boundary in best_boundaries[:]:
        
    
        
#         boundaries = calculate_boundaries(df, discretize_column)
#         splits = [(df.iloc[0:x+1,:], df.iloc[x+1:,:], y) for x,y in enumerate(boundaries)]
#         gains = []
#         for s1,s2, boundary in splits:
#             probs_s1 =  calculate_prob(s1[discretize_column])
#             probs_s2 =  calculate_prob(s2[discretize_column])
#             entropy_s1 = stats.entropy(probs_s1, base=2)
#             entropy_s2 = stats.entropy(probs_s2, base=2)
#             total_size = df.shape[0]
#             gains.append((info_gain(entropy_s1,entropy_s2,s1,s2,total_size) , boundary,s1, s2))

         
#         best_boundaries.append(min(gains))
#         end_result.append(min(gains))
#         current_bins += 1
#         if current_bins == nbins:
#             print("bin limit reached end it")
#             return 
        
    
    
    
    

   
        
# #         print("gain",gain)
#         print("probs1", probs_s1)
#         print("probs2", probs_s2)
        
#         print("entropy_s1" ,entropy_s1)
#         print("entropy_s2",entropy_s2)
#         print("boundary",boundary)
#         print("s1",s1)
#         print("s2",s2)
#     print(gains
#     print(f"Current best cut is {min(gains)[1]} with gain {min(gains)[0]}")
#     print("S1")
#     print(min(gains)[2])
#     print("S2")
#     print(min(gains)[3])
#     print(best_boundaries)
mpg =pd.read_csv("Datasets/mpg.csv")
mpg = mpg[['displ','class']]
print(mpg.shape)
# print(mpg.head())

result = entropy_discretize(df = wah, discretize_column = 'hours_studied',class_col = 'grade', nbins = 6)
for a,b,c,d in result:
    print(f"Info gain: {a}")
    print(f"Boundary : {b}")
    print("")
    print("Set 1")
    print(c.head())
    print("Set 2")
    print(d.head())
    print("-" * 30)
    print("")
    
    
    

(234, 2)
Info gain: 1.1019550008653873
Boundary : 10.5

Set 1
   hours_studied grade
0              4     N
1              5     Y
2              9     N
Set 2
   hours_studied grade
3             12     Y
4             15     Y
------------------------------

Info gain: 0.8
Boundary : 4.5

Set 1
   hours_studied grade
0              4     N
Set 2
   hours_studied grade
1              5     Y
2              9     N
------------------------------

Info gain: 0.0
Boundary : 13.5

Set 1
   hours_studied grade
3             12     Y
Set 2
   hours_studied grade
4             15     Y
------------------------------



### Make boundary function

In [7]:
# calculate boundaries for a continous variable
# vectorized
def calculate_boundaries(df = None, column = None):
    if df is None:
        raise ValueError("dataframe must not be empty")
    if column is None:
        raise ValueError("column must not be empty")
    if is_numeric_dtype(df[column]) != True:
        raise TypeError("column must be of numeric type")

    return pd.Series((df[column][1:].values + df[column][:-1].values) / 2, name = "boundaries")  


In [8]:
boundaries = calculate_boundaries(agey, 'age')
print(boundaries)

0    21.5
1    23.0
2    24.5
3    26.0
4    27.0
5    27.0
6    31.0
7    38.0
Name: boundaries, dtype: float64


In [82]:
# discretizes a column with respect to target column that contains the class to be classified
# def entropy_discretize(df = None, target_column = None, discretize_column ):
#     boundaries = calculate_boundaries(df, 'age')

# split dataframe
for x,y in enumerate(boundaries):
    print(f"Next Split with boundary {y}")
    print("-" * 30)
    print("_" * 30)
    print("S1")
    print(agey.iloc[0:x+1,:])
    print("_" * 30)
    print("S2")
    print(agey.iloc[x+1:,:])
    print("\n")

# print(agey.iloc[0:2,:])
# print(agey.iloc[1+1:,:])
 
# for x,y in enumerate(agey['age']):
#         if x == (agey['age'].size - 1):
#             break
#         else:
#           print(x,x+1)
# print(agey['age'])
# print(calculate_boundaries(agey, 'age'))

Next Split with boundary 21.5
------------------------------
______________________________
S1
   age grade
0   21     F
______________________________
S2
   age grade
1   22     F
2   24     P
3   25     F
4   27     P
5   27     P
6   27     P
7   35     P
8   41     P


Next Split with boundary 23.0
------------------------------
______________________________
S1
   age grade
0   21     F
1   22     F
______________________________
S2
   age grade
2   24     P
3   25     F
4   27     P
5   27     P
6   27     P
7   35     P
8   41     P


Next Split with boundary 24.5
------------------------------
______________________________
S1
   age grade
0   21     F
1   22     F
2   24     P
______________________________
S2
   age grade
3   25     F
4   27     P
5   27     P
6   27     P
7   35     P
8   41     P


Next Split with boundary 26.0
------------------------------
______________________________
S1
   age grade
0   21     F
1   22     F
2   24     P
3   25     F
__________________

## Make probability function 
Entropy function
$$\large  ENT(S_1) = - \sum_{i=1}^mp_ilog_2(p_i)$$

In [10]:
# probablity calculation
# vectorized usin numpy
def calculate_prob(column_series = None):
    if column_series is None:
        raise ValueError("Series column must not be empty")
    if isinstance(column_series, pd.Series) != True:
        raise TypeError("Series column must be a series")
#     if is_string_dtype(column_series) == False and is_categorical_dtype(column_series) == False :
#         raise TypeError("Series column must be of string or numerice type to be a class")
    total_observations = column_series.size
    unique_counts = np.unique(column_series, return_counts = True)
    probs = unique_counts[1] / total_observations
    return probs


## Calculate entropy using probabilities

In [94]:
print(agey['grade'])
# calculate probability of each class
probs =  calculate_prob(agey['grade'])
print(probs)
entropy = stats.entropy(probs, base=2)
print(entropy)

0    F
1    F
2    P
3    F
4    P
5    P
6    P
7    P
8    P
Name: grade, dtype: object
[0.33333333 0.66666667]
0.9182958340544894


In [20]:
def info_gain(e1,e2,s1,s2,total_size):
    return ((s1.size/total_size) * e1) + ((s2.size/total_size) * e2)


calc entropy

for each best-boundary
split set into majorsplit

for each 

calc boundaries

split set via boundaries into set1 and set2 i.e (s1,s2,boundary)
for each boundary:
    find entropy of set1 and set2
    find gain 
    store gain and boundary
find and store best-boundary with max-gain
check end cond (either gain of number of bins i.e best-boundaries (needs more explicit definition)
if true terminate else continue (loop to beginning)  <-- the recursive part


